# Loading the Census Income (KDD) Dataset
The `.names` file is a convention used by the UCI Machine Learning Repository to provide essential metadata about the datasets they host.

In [2]:
import time

# Importing necessary objects for data manipulation and analysis.
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, precision_recall_curve, auc, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

In [3]:
# It's easier to extract useful infomration about column names and distinct values from the .names file then to write it out.
def parse_names():
    """
    Parses the .names file from the Census-Income (KDD) dataset to extract information about the variables.

    Returns:
        dict: A dictionary where each key is a variable name, and the value describes the variable as either 'continuous' or 'nominal'.
              For nominal variables, it includes the distinct values found in the dataset.
    """

    result = {}

    # The .names file doesn't follow a strict format.
    with open('../data/census/census-income.names', 'r') as file:
        text = file.read()

    # Just take lines that define attributes.
    etch = list(filter(lambda x: not (x.startswith('|') or x.startswith('-') or x.strip() == ''), text.splitlines()))
    for line in etch:
        key, value = map(lambda x: x.rstrip(".").strip(), line.split(':'))

        if value == "continuous":
            result[key] = {"type": "continuous"}
        else:
            values = list(map(lambda x: x.strip(), value.split(",")))

            # Try converting to an integer.
            try:
                values = list(map(int, values))
            except:
                pass
            result[key] = {"type": "nominal", "values": values}

    return result

In [4]:
# Invoke the built-in help system.
help(parse_names)

Help on function parse_names in module __main__:

parse_names()
    Parses the .names file from the Census-Income (KDD) dataset to extract information about the variables.
    
    Returns:
        dict: A dictionary where each key is a variable name, and the value describes the variable as either 'continuous' or 'nominal'.
              For nominal variables, it includes the distinct values found in the dataset.



In [5]:
# A 'codebook' is just something to help with understanding a dataset.
codebook = parse_names()

# The target is the last column in the dataset.
column_names = list(codebook.keys()) + ["income"]

In [6]:
train_df = pd.read_csv('../data/census/census-income.data', names=column_names, index_col=False)
train_df.shape

(199523, 42)

In [7]:
test_df = pd.read_csv('../data/census/census-income.test', names=column_names, index_col=False)
test_df.shape

(99762, 42)

# Exploring the Dataset

Pandas preserves spaces that are in the dataset, these spaces aren't important and can be removed while exploring.

Missing values show up in the dataset as '?'.

In [9]:
df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# You can check the column names, missing values, and data type.
df.replace('?', None).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 42 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   age                                         199523 non-null  int64  
 1   class of worker                             199523 non-null  object 
 2   detailed industry recode                    199523 non-null  int64  
 3   detailed occupation recode                  199523 non-null  int64  
 4   education                                   199523 non-null  object 
 5   wage per hour                               199523 non-null  int64  
 6   enroll in edu inst last wk                  199523 non-null  object 
 7   marital stat                                199523 non-null  object 
 8   major industry code                         199523 non-null  object 
 9   major occupation code                       199523 non-null  object 
 

In [10]:
# Find values that aren't listed in .names.
def display_unexpected_values(df, codebook):
    for key, value in codebook.items():
        if value["type"] == "continuous":
            continue

        expected_values = set(value["values"])
        actual_values = set(df[key].unique())
        unexpected_values = actual_values - expected_values

        if unexpected_values:
            print(f"{key}: {unexpected_values}")

# The .names file doens't talk about '?' as a posible value for the nominals which is why it's showing up.
display_unexpected_values(df, codebook)

state of previous residence: {'?'}
migration code-change in msa: {'?'}
migration code-change in reg: {'?'}
migration code-move within reg: {'?'}
migration prev res in sunbelt: {'?'}
country of birth father: {'?'}
country of birth mother: {'?'}
country of birth self: {'?'}


In [11]:
def calculate_missingness(df):
    for column in df.select_dtypes(include=['object']):
        missing_count = (df[column].str.strip() == "?").sum()
        total_count = len(df[column])
        missing_percentage = (missing_count / total_count) * 100
        if missing_count > 0:
            print(f"{column}: {missing_percentage:.2f}% missing")

# Computes the percentage of missing values represented by '?' for string columns in the dataset.
calculate_missingness(train_df)

state of previous residence: 0.35% missing
migration code-change in msa: 49.97% missing
migration code-change in reg: 49.97% missing
migration code-move within reg: 49.97% missing
migration prev res in sunbelt: 49.97% missing
country of birth father: 3.36% missing
country of birth mother: 3.07% missing
country of birth self: 1.70% missing


In [12]:
# Let's only look at columns that actually represent numeric data.
continuous = list(filter(lambda x: codebook[x]["type"] == "continuous" and x != "instance weight", codebook.keys()))

# Some of these values might be capped.
df[continuous].describe().T[["min", "max"]]

,min,max
age,0.0,90.0
wage per hour,0.0,9999.0
capital gains,0.0,99999.0
capital losses,0.0,4608.0
dividends from stocks,0.0,99999.0
num persons worked for employer,0.0,6.0
weeks worked in year,0.0,52.0


# Binary Classification Task

The **Census-Income (KDD)** dataset is derived from the **1994 and 1995 Current Population Surveys (CPS)** conducted by the U.S. Census Bureau.

It includes **199,523 training instances** and **99,762 test instances**.

The goal is to predict the income level based on the data provided in each row. Incomes have been binned at the $50K level to create a binary classification problem.

In [14]:
# List of categorical columns.
nominal = list(filter(lambda x: codebook[x]["type"] == "nominal", codebook.keys()))

# List of numerical columns.
continuous = list(filter(lambda x: codebook[x]["type"] == "continuous", codebook.keys()))

# Ignore 'instance weight'.
continuous.remove("instance weight")

# The 'instance weight' column happens to make the RandomForestClassifier more accurate.
# The dataset feels like a distribution problem to me instead of a prediction problem so I'd tend to use any columns that help with modeling the distribution.
# The .names file suggested to not use it... I don't even think there are real realtionships to learn in this data though.


## Encoding Categorical Features

The way features are encoded for this dataset doesn't seem to make much difference in the modeling.

Most columns do not have a natural ordering and should use One-Hot encoding for best performance!

The `OneHotEncoder` makes things substantially slower though, so I need to suffer through that.

In [97]:
# education_order = [
#     ' Children',
#     ' Less than 1st grade',
#     ' 1st 2nd 3rd or 4th grade',
#     ' 5th or 6th grade',
#     ' 7th and 8th grade',
#     ' 9th grade',
#     ' 10th grade',
#     ' 11th grade',
#     ' 12th grade no diploma',
#     ' High school graduate',
#     ' Some college but no degree',
#     ' Associates degree-occup /vocational',
#     ' Associates degree-academic program',
#     ' Bachelors degree(BA AB BS)',
#     ' Masters degree(MA MS MEng MEd MSW MBA)',
#     ' Doctorate degree(PhD EdD)',
#     ' Prof school degree (MD DDS DVM LLB JD)'
# ]

# ordinal_encoder = OrdinalEncoder(categories=[education_order])
# ordinal_encoder.fit(train_df[['education']])

# education_df = pd.DataFrame(ordinal_encoder.transform(train_df[['education']]), columns=['education'])
# education_test_df = pd.DataFrame(ordinal_encoder.transform(test_df[['education']]), columns=['education'])

In [16]:
# Encode categorical features.
encoder = OneHotEncoder(drop='first', sparse_output=False)
label_encoder = LabelEncoder()

# Fit the encoders to the training data.
encoder.fit(train_df[nominal])
label_encoder.fit(train_df["income"])

# Names for the encoded features.
feature_names = encoder.get_feature_names_out(nominal)

In [17]:
# The unique values are sorted in lexicographical order.
label_encoder.classes_

array([' - 50000.', ' 50000+.'], dtype=object)

In [115]:
# Encode training data.
X = pd.concat([pd.DataFrame(encoder.transform(train_df[nominal]), columns=feature_names), train_df[continuous]], axis=1)
y = label_encoder.transform(train_df["income"])


In [117]:
# Encode testing data.
X_test = pd.concat([pd.DataFrame(encoder.transform(test_df[nominal]), columns=feature_names), test_df[continuous]], axis=1)
y_test = label_encoder.transform(test_df["income"])


## Scaling Features

You don't need to scale the categorical columns.

In [103]:
# Fit a standard scaler to training data.
scaler = StandardScaler()
scaler.fit(X[continuous])

# Transform the training data.
X[continuous] = scaler.transform(X[continuous])

# Transform the testing data.
X_test[continuous] = scaler.transform(X_test[continuous])

## Evaluating a Model

The RandomForestClassifier is relatively robust, as it isn't significantly affected by how features are encoded or scaled, and it shows some resistance to irrelevant columns as well.

In [105]:
# Fit with training data.
model = RandomForestClassifier(random_state=42)
model.fit(X, y);


In [107]:
# Predict with testing data.
predictions = model.predict(X_test)

In [109]:
# The correct labels should come first. I had flipped these before giving me bad looking results.
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     93576
           1       0.73      0.40      0.52      6186

    accuracy                           0.95     99762
   macro avg       0.85      0.70      0.75     99762
weighted avg       0.95      0.95      0.95     99762



In [111]:
print(balanced_accuracy_score(predictions, y_test))

0.8468403146414989


In [113]:
# Measures the model's ability to distinguish between classes.
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

0.9367346843267779

In [28]:
precision, recall, thresholds = precision_recall_curve(y_test, model.predict_proba(X_test)[:, 1])

# Measures the model's ability to balance precision and recall across different thresholds.
auc(recall, precision)

0.6246328289239371

In [29]:
predict_probabilities = model.predict_proba(X_test)[:, 1]

for threshold in np.arange(0.1, 1.0, 0.1):
    predictions = (predict_probabilities >= threshold).astype(int)

    # Calculate precision and recall.
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)

    print(f'Threshold: {threshold:.2f} | Precision: {precision:.4f} | Recall: {recall:.4f}')

Threshold: 0.10 | Precision: 0.3222 | Recall: 0.8590
Threshold: 0.20 | Precision: 0.4556 | Recall: 0.7449
Threshold: 0.30 | Precision: 0.5699 | Recall: 0.6107
Threshold: 0.40 | Precision: 0.6472 | Recall: 0.5124
Threshold: 0.50 | Precision: 0.7298 | Recall: 0.4100
Threshold: 0.60 | Precision: 0.8055 | Recall: 0.2913
Threshold: 0.70 | Precision: 0.8670 | Recall: 0.1866
Threshold: 0.80 | Precision: 0.9103 | Recall: 0.1115
Threshold: 0.90 | Precision: 0.9450 | Recall: 0.0445


## Dealing With Class Imbalance

Oversampling can improve the reseults on the minority class slightly and it doesn't take a long time to run.

In [31]:
def display_class_imbalance(target):
    class_0_count = np.count_nonzero(target == 0)
    class_1_count = np.count_nonzero(target == 1)
    total_count = len(target)

    # Calculate the percentages.
    class_0_percentage = (class_0_count / total_count) * 100
    class_1_percentage = (class_1_count / total_count) * 100

    print(f"Class 0: {class_0_count} instances, {class_0_percentage}% of the total")
    print(f"Class 1: {class_1_count} instances, {class_1_percentage}% of the total")


In [32]:
display_class_imbalance(y_test)

Class 0: 93576 instances, 93.7992421964275% of the total
Class 1: 6186 instances, 6.200757803572503% of the total


In [33]:
# Resampling takes a while this function let's me check how long.
def resample(X, y, sampler_class, random_state=42):
    start_time = time.time()
    sampler = sampler_class(random_state=random_state)
    X_resampled, y_resampled = sampler.fit_resample(X, y)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{elapsed_time:.2f} seconds")
    return X_resampled, y_resampled

#### Random Oversampling

In [35]:
X_ros, y_ros = resample(X, y, RandomOverSampler)

0.62 seconds


#### Random Undersampling

In [37]:
X_rus, y_rus = resample(X, y, RandomUnderSampler)

0.06 seconds


#### Cluster Centroids

In [39]:
X_cc, y_cc = resample(X, y, ClusterCentroids)

627.34 seconds


#### Synthetic Minority Oversampling Technique (SMOTE)

In [41]:
X_smote, y_smote = resample(X, y, SMOTE)

2.49 seconds


#### SMOTE and Edited Nearest Neighbors (SMOTEENN)

In [43]:
X_smoteenn, y_smoteenn = resample(X, y, SMOTEENN)

370.05 seconds


### Building and Evaluating Models on Resampled Data

In [45]:
model_ros = RandomForestClassifier(random_state=42)
model_ros.fit(X_ros, y_ros);

In [46]:
model_rus = RandomForestClassifier(random_state=42)
model_rus.fit(X_rus, y_rus);

In [47]:
model_cc = RandomForestClassifier(random_state=42)
model_cc.fit(X_cc, y_cc);

In [48]:
model_smote = RandomForestClassifier(random_state=42)
model_smote.fit(X_smote, y_smote);

In [49]:
model_smoteenn = RandomForestClassifier(random_state=42)
model_smoteenn.fit(X_smoteenn, y_smoteenn);

In [50]:
models = [
    (model_rus, "Random Oversampling"),
    (model_ros, "Random Undersampling"),
    (model_cc, "Cluster Centroids"),
    (model_smote, "SMOTE"),
    (model_smoteenn, "SMOTEENN")
]

for model, text in models:
    print(f"{text}: ")
    predictions = model.predict(X_test)
    print(classification_report(predictions, y_test))
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, predictions)}\n")

Random Oversampling: 
              precision    recall  f1-score   support

           0       0.85      0.99      0.91     79746
           1       0.90      0.28      0.42     20016

    accuracy                           0.85     99762
   macro avg       0.87      0.63      0.67     99762
weighted avg       0.86      0.85      0.81     99762

Balanced Accuracy: 0.871470479925258

Random Undersampling: 
              precision    recall  f1-score   support

           0       0.98      0.97      0.97     94825
           1       0.50      0.62      0.55      4937

    accuracy                           0.95     99762
   macro avg       0.74      0.79      0.76     99762
weighted avg       0.96      0.95      0.95     99762

Balanced Accuracy: 0.7380789440319241

Cluster Centroids: 
              precision    recall  f1-score   support

           0       0.49      1.00      0.66     45866
           1       0.98      0.11      0.20     53896

    accuracy                           0